In [38]:
import csv
import plotly.graph_objects as go
from extra import ConfidenceInterval
import pandas as pd

In [39]:
circuitlength_EHV = 2471  # unit: km ## Q2: 2310, Q3: 2471
circuitlength_HV = 4078  # unit: km ## Q2: 3329, Q3: 4078
failures_EHV = 42  # Q2: 25, Q3: 42
failures_HV = 167  # Q2: 51, Q3: 167
time = 8  # unit: years

alpha = 0.05

## Calculate the bounds of the confidence interval of EHV statistics
lower_bound_EHV, upper_bound_EHV = ConfidenceInterval(
    alpha, failures_EHV, circuitlength_EHV, time
)
## Calculate the bounds of the confidence interval of HV statistics
lower_bound_HV, upper_bound_HV = ConfidenceInterval(
    alpha, failures_HV, circuitlength_HV, time
)
## Calculate the bounds of the confidence interval of HV statistics
lower_bound_OHL, upper_bound_OHL = ConfidenceInterval(
    alpha, failures_EHV + failures_HV, circuitlength_EHV + circuitlength_HV, time
)

print("EHV confidence interval:", lower_bound_EHV, upper_bound_EHV)
print("HV confidence interval:", lower_bound_HV, upper_bound_HV)
print("OHL confidence interval:", lower_bound_OHL, upper_bound_OHL)
# boxplot()

f_EHV = failures_EHV / (circuitlength_EHV * time)
f_HV = failures_HV / (circuitlength_HV * time)
f_OHL = (failures_EHV + failures_HV) / ((circuitlength_EHV + circuitlength_HV) * time)

EHV confidence interval: 0.0015312578780087868 0.0028719040256217454
HV confidence interval: 0.004371987446835723 0.005956888553127153
OHL confidence interval: 0.0034666330612644143 0.0045682079221547635


In [40]:
# Pandas dataframe with columns: 'type','value' ,'lower bound', 'upper bound' 
df = pd.DataFrame(
    {
        "type": ["EHV", "HV", "OHL"],
        "value": [f_EHV, f_HV, f_OHL],
        "lower bound": [lower_bound_EHV, lower_bound_HV, lower_bound_OHL],
        "upper bound": [upper_bound_EHV, upper_bound_HV, upper_bound_OHL],
    }
)
df['rounded_value'] = df['value'].round(4)
df

,type,value,lower bound,upper bound,rounded_value
0,EHV,0.002125,0.001531,0.002872,0.0021
1,HV,0.005119,0.004372,0.005957,0.0051
2,OHL,0.003989,0.003467,0.004568,0.0040


In [41]:
# Plot a point with error bars for each type and the confidence interval
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df["type"],
        y=df["value"],
        error_y=dict(
            type="data",
            symmetric=False,
            array=df["upper bound"] - df["value"],
            arrayminus=df["value"] - df["lower bound"],
        ),
        mode="markers+text",
        marker=dict(color="blue", size=12),
        text=df["rounded_value"],
    )
)
# Set labels position (text position)
fig.update_traces(textposition='top center')
# Increase font size of text for datapoints
# Size 800x800
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    title="Failure frequency for 8 years",
    xaxis_title="Type of line",
    yaxis_title="Failure frequency",
)
# Add label to each point

# Show figure
fig.show()
fig.write_html('./plots/Q3.html')
fig.write_image('./plots/Q3.png')